In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install basicsr realesrgan pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.

In [ ]:
pip install torchvision.transforms.functional_tensor

ERROR: Could not find a version that satisfies the requirement torchvision.transforms.functional_tensor (from versions: none)
ERROR: No matching distribution found for torchvision.transforms.functional_tensor


In [ ]:
import torch
from torchvision.transforms.functional import rgb_to_grayscale
import numpy as np
from PIL import Image
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer

model_path = "/content/drive/MyDrive/RealESRGAN_x4plus.pth" #chemin au modèle

state_dict = torch.load(model_path, map_location=torch.device('cpu'))['params_ema']

model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
model.load_state_dict(state_dict, strict=True)

upsampler = RealESRGANer(
    scale = 4,
    model_path= model_path,
    model = model,
    tile = 0,
    pre_pad = 0,
    half = True
)

img = Image.open("image_test.jpg").convert('RGB') # chemin au image à tester
img = np.array(img)

output, _ = upsampler.enhance(img, outscale=4)

output_img = Image.fromarray(output)
output_img.save("test_img_x4.png")

<ipython-input-18-bfd47e38a628>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location=torch.device('cpu'))['params_ema']
/usr/lo

In [ ]:
import os
import torch
import numpy as np
from PIL import Image
from torchvision.transforms.functional import rgb_to_grayscale
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer

def process_images(base_folder="/content/drive/MyDrive/data_diffusion"):
    # Dossier de sortie (ne sera pas parcouru pour le traitement)
    output_base = os.path.join(base_folder, "data_prep")
    images_out = os.path.join(output_base, "images")
    desc_out = os.path.join(output_base, "descriptions")

    # Création des dossiers de sortie
    os.makedirs(images_out, exist_ok=True)
    os.makedirs(desc_out, exist_ok=True)

    # Chargement du modèle RealESRGAN
    model_path = "/content/drive/MyDrive/RealESRGAN_x4plus.pth"
    try:
        state_dict = torch.load(model_path, map_location=torch.device('cpu'))['params_ema']
    except Exception as e:
        print(f"Erreur lors du chargement du modèle : {e}")
        return

    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
    try:
        model.load_state_dict(state_dict, strict=True)
    except Exception as e:
        print(f"Erreur lors du chargement du state_dict dans le modèle : {e}")
        return

    upsampler = RealESRGANer(
        scale=4,
        model_path=model_path,
        model=model,
        tile=0,
        pre_pad=0,
        half=True
    )

    # Compteur global pour renommer les images de façon séquentielle
    global_counter = 1

    # Parcours de tous les sous-dossiers dans "data_diffusion" en ignorant "data_prep"
    for sub in os.listdir(base_folder):
        sub_path = os.path.join(base_folder, sub)
        if sub == "data_prep" or not os.path.isdir(sub_path):
            continue

        images_dir = os.path.join(sub_path, "images")
        desc_dir = os.path.join(sub_path, "descriptions")
        if not os.path.exists(images_dir):
            print(f"Le dossier 'images' n'existe pas dans {sub_path}.")
            continue
        if not os.path.exists(desc_dir):
            print(f"Le dossier 'descriptions' n'existe pas dans {sub_path}.")
            continue

        for file in os.listdir(images_dir):
            if file.lower().endswith('.jpg'):
                try:
                    # Construction du chemin de l'image source
                    image_path = os.path.join(images_dir, file)
                    # Ouverture, conversion et redimensionnement de l'image
                    with Image.open(image_path) as img:
                        img = img.convert('RGB')
                        img = img.resize((256, 256))
                        img_array = np.array(img)

                    # Application de l'amélioration via RealESRGAN
                    output, _ = upsampler.enhance(img_array, outscale=4)
                    output_image = Image.fromarray(output)

                    # Création du nouveau nom d'image en utilisant le compteur global
                    new_image_name = f"image_{global_counter}.jpg"
                    new_image_path = os.path.join(images_out, new_image_name)
                    output_image.save(new_image_path)
                    print(f"Image traitée et sauvegardée : {new_image_path}")

                    # Traitement du fichier de description associé
                    # On cherche le fichier de description correspondant à l'image source.
                    # On suppose que le nom de la description correspond à l'image (extension .txt)
                    desc_filename = os.path.splitext(file)[0] + ".txt"
                    desc_path = os.path.join(desc_dir, desc_filename)
                    if os.path.exists(desc_path):
                        new_desc_name = f"image_{global_counter}.txt"
                        new_desc_path = os.path.join(desc_out, new_desc_name)
                        try:
                            with open(desc_path, 'r', encoding='utf-8') as f_in:
                                desc_content = f_in.read()
                            with open(new_desc_path, 'w', encoding='utf-8') as f_out:
                                f_out.write(desc_content)
                            print(f"Description copiée : {new_desc_path}")
                        except Exception as desc_err:
                            print(f"Erreur lors de la copie de la description {desc_filename}: {desc_err}")
                    else:
                        print(f"Aucune description trouvée pour l'image {file}")

                    # Incrémentation du compteur pour le prochain fichier
                    global_counter += 1
                except Exception as e:
                    print(f"Erreur lors du traitement du fichier {file} dans {sub_path} : {e}")

if __name__ == '__main__':
    process_images()


Image traitée et sauvegardée : /content/drive/MyDrive/data_diffusion/data_prep/images/image_1.jpg
Description copiée : /content/drive/MyDrive/data_diffusion/data_prep/descriptions/image_1.txt
Image traitée et sauvegardée : /content/drive/MyDrive/data_diffusion/data_prep/images/image_2.jpg
Description copiée : /content/drive/MyDrive/data_diffusion/data_prep/descriptions/image_2.txt
Image traitée et sauvegardée : /content/drive/MyDrive/data_diffusion/data_prep/images/image_3.jpg
Description copiée : /content/drive/MyDrive/data_diffusion/data_prep/descriptions/image_3.txt
Image traitée et sauvegardée : /content/drive/MyDrive/data_diffusion/data_prep/images/image_4.jpg
Description copiée : /content/drive/MyDrive/data_diffusion/data_prep/descriptions/image_4.txt
Image traitée et sauvegardée : /content/drive/MyDrive/data_diffusion/data_prep/images/image_5.jpg
Description copiée : /content/drive/MyDrive/data_diffusion/data_prep/descriptions/image_5.txt
Image traitée et sauvegardée : /content/